In [10]:
from tqdm import tqdm
from glob import glob
import tifffile
import numpy as np
import os
from EmbedSeg.utils.preprocess_data import extract_data, split_train_val, get_data_properties
from EmbedSeg.utils.generate_crops import *
import json

### Download Data

In [11]:
data_dir = '../../../data'
project_name = 'custom'

Ideally, <b>*.tif</b>-type images and the corresponding masks should be respectively present under <b>images</b> and <b>masks</b>, under directories <b>train</b>, <b>val</b> and <b>test</b>, which can be present at any location on your workstation, pointed to by the variable <i>data_dir</i>. (In order to prepare such instance masks, one could use the Fiji plugin <b>Labkit</b> as detailed <b>[here](https://github.com/juglab/EmbedSeg/wiki/Use-Labkit-to-prepare-instance-masks)</b>). The following would be the desired structure as to how data should be present. 

<img src="../../../static/png/01_dir_structure.png" width="100"/>

If you already have your data available in the above style, please skip to the <b><a href="#center">third</a></b> section of this notebook, where you specify the kind of center to which constitutive pixels of an object should embed. 
Since for the <b> dsb-2018</b> dataset, we do not have the data in this format yet, we firstly download the data from an external url in the following cells, next we split this data to create our `train`, `val` and `test` directories. 

The images and corresponding masks are downloaded from an external url, specified by `zip_url` to the path specified by the variables `data_dir` and `project_name`. The following structure is generated after executing the `extract_data` and `split_train_val` methods below:

<img src="../../../static/png/03_dsb-2018.png" width="400"/>

### Split Data into `train`, `val` \& `test`

Now, we would like to reserve a small fraction (15 % by default) of the provided train dataset as validation data. Here, in case you would like to repeat multiple experiments with the same partition, you may continue and press <kbd>Shift</kbd> + <kbd>Enter</kbd> on the next cell - but in case, you would like different partitions each time, please add the `seed` attribute equal to a different integer (For example, 
```
split_train_val(
data_dir = data_dir, 
project_name = project_name, 
train_val_name = 'train', 
subset = 0.15, 
seed = 1000)
```
)

In [13]:
split_train_val(
    data_dir = data_dir,
    project_name = project_name, 
    train_val_name = 'train',
    subset = 0.15)

Created new directory : ../../../data\custom\train\images/
Created new directory : ../../../data\custom\train\masks/
Created new directory : ../../../data\custom\val\images/
Created new directory : ../../../data\custom\val\masks/
Created new directory : ../../../data\custom\test\images/
Created new directory : ../../../data\custom\test\masks/
Train-Val-Test Images/Masks copied to ../../../data\custom


### Specify desired centre location for spatial embedding of pixels

Interior pixels of an object instance can either be embedded at the `medoid`, the `approximate-medoid` or the `centroid`. 

In [14]:
center = 'medoid' # 'medoid', 'approximate-medoid', 'centroid'
try:
    assert center in {'medoid', 'approximate-medoid', 'centroid'}
    print("Spatial Embedding Location chosen as : {}".format(center))
except AssertionError as e:
    e.args += ('Please specify center as one of : {"medoid", "approximate-medoid", "centroid"}', 42)
    raise



Spatial Embedding Location chosen as : medoid


### Calculate some dataset specific properties

In the next cell, we will calculate properties of the data such as `min_object_size`, `foreground_weight` etc. <br>
We will also specify some properties, for example,  
* set `one_hot = True` in case the instances are encoded in a one-hot style. 
* set `data_properties_dir['data_type']='16-bit'` if the images are of datatype `unsigned 16 bit` and 
    `data_properties_dir['data_type']='8-bit'` if the images are of datatype `unsigned 8 bit`.

Lastly, we will save the dictionary `data_properties_dir` in a json file `data_properties.json`, which we will access in the `02-train` and `03-predict` notebooks.

In [15]:
one_hot = False
data_properties_dir = get_data_properties(data_dir, project_name, train_val_name=['train'], 
                                          test_name=['test'], mode='2d', one_hot=one_hot)

data_properties_dir['data_type']='16-bit'

with open('data_properties.json', 'w') as outfile:
    json.dump(data_properties_dir, outfile)
    print("Dataset properies of the `{}` dataset is saved to `data_properties.json`".format(project_name))

Calculating foreground weight...


100%|██████████| 165/165 [00:00<00:00, 186.86it/s]


    Foreground weight of the `custom` dataset set equal to 10.000
Calculating minimum object size...


100%|██████████| 165/165 [00:02<00:00, 75.48it/s] 


    Minimum object size of the `custom` dataset is equal to 4
Calculating max_eval_image_size...


100%|██████████| 69/69 [00:00<00:00, 193.82it/s]


Maximum evaluation image size of the `custom` dataset set equal to (1024, 1024)
Calculating avg_background_intensity...


100%|██████████| 165/165 [00:01<00:00, 138.42it/s]

    Average background intensity of the `custom` dataset set equal to 10499.785
Dataset properies of the `custom` dataset is saved to `data_properties.json`


### Specify cropping configuration parameters

Images and the corresponding masks are cropped into patches centred around an object instance, which are pre-saved prior to initiating the training. Here, `data_subsets` is a list of names of directories which is processed. Note that the cropped images, masks and center-images would be saved at the path specified by `crops_dir` (The parameter `crops_dir` is set to ```./crops``` by default, which creates a directory at the same location as this notebook). 

In [20]:
crops_dir = './crops'
data_subsets = ['train', 'val'] 
crop_size = 128

### Generate Crops



<div class="alert alert-block alert-warning"> 
    The cropped images and masks are saved at the same-location as the example notebooks. <br>
    Generating the crops would take a little while!
</div>

In [21]:
for data_subset in data_subsets:
    image_dir = os.path.join(data_dir, project_name, data_subset, 'images')
    instance_dir = os.path.join(data_dir, project_name, data_subset, 'masks')
    image_names = sorted(glob(os.path.join(image_dir, '*.tif'))) 
    instance_names = sorted(glob(os.path.join(instance_dir, '*.tif')))  
    for i in tqdm(np.arange(len(image_names))):
        if one_hot:
            process_one_hot(image_names[i], instance_names[i], os.path.join(crops_dir, project_name), data_subset, crop_size, center, one_hot = one_hot)
        else:
            process(image_names[i], instance_names[i], os.path.join(crops_dir, project_name), data_subset, crop_size, center, one_hot=one_hot)
    print("Cropping of images, instances and centre_images for data_subset = `{}` done!".format(data_subset))

100%|██████████| 165/165 [07:28<00:00,  2.72s/it]


Cropping of images, instances and centre_images for data_subset = `train` done!


100%|██████████| 28/28 [01:06<00:00,  2.37s/it]

Cropping of images, instances and centre_images for data_subset = `val` done!
